# quick inference poc

### Imports

In [1]:
# -- public imports

from transformers import BigBirdTokenizer, AutoTokenizer, AutoModelForTokenClassification, AutoConfig
import pandas as pd
from torch.utils.data import DataLoader
import torch

c:\Users\viksp\Anaconda3\envs\argumin\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.big_bird because of the following error (look up to see its traceback):
cannot import name '_sentencepiece' from partially initialized module 'sentencepiece' (most likely due to a circular import) (c:\Users\viksp\Anaconda3\envs\argumin\lib\site-packages\sentencepiece\__init__.py)

In [ ]:
# -- private import
from argminer.data import KaggleDataset

: 

In [ ]:
# -- dev imports

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
model = 'google/bigbird-roberta-large'

tokenizer = AutoTokenizer.from_pretrained(model, add_prefix_space=True)
config_model = AutoConfig.from_pretrained(model) 
config_model.num_labels = 3
model = AutoModelForTokenClassification.from_pretrained(model, config=config_model)
optimizer = torch.optim.Adam(params=model.parameters())

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
Some weights of the model checkpoint at google/bigbird-roberta-large were not used when initializing BigBirdForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a Ber

In [32]:
texts = [
            'Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble!',
            'Thing about! '
        ]
labels = [
    ['O', 'O', 'O', 'O', 'B-PERS', 'I-PERS'],
    ['O', 'O', 'O', 'B-PERS']
]

# TODO see if bert can accept text inputs?
labels_numeric = [
    [0, 0, 0, 0, 1, 2],
    [0, 0, 0, 1]
]

In [33]:
df = pd.DataFrame({
        'text': texts,
        'labels': labels_numeric
    })

In [36]:
dataset = KaggleDataset(
    df,
    tokenizer,
    max_length=20
)

loader = torch.utils.data.DataLoader(dataset, batch_size=2)

In [71]:
for i, (inputs, targets) in enumerate(loader):
    print(inputs)

    optimizer.zero_grad()
    
    # for evaluation purposes may need to save the embedding weights??
    
    word_ids = inputs['word_ids']
    word_id_mask = inputs['word_id_mask']
    loss, outputs = model(
        labels=targets,
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        return_dict=False
    )
    print(outputs)
    inference(outputs, word_ids, 'avg')
    break
    

{'input_ids': tensor([[   65, 16003,   717,  1539,   419,   676,   617,   992,   500,  6378,
            66,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [   65, 16003,   415,   817,  1012, 47489,    66,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'word_ids': tensor([[-1,  0,  1,  2,  3,  4,  4,  4,  5,  5, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1],
        [-1,  0,  1,  2,  3,  3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1]]), 'word_id_mask': [tensor([False, False]), tensor([True, True]), tensor([True, True]), tensor([True, True]), tensor([True, True]), tensor([True, True]), tensor([ True, False]), tensor([ True, False]), tensor([ True, False]), tensor([ True, False]), tensor([False, False]), tensor([False, False]),

In [70]:
def inference(batch, word_id_batch, agg_strategy):
    # TODO need to have assertions to prevent certain strategies from happening together!


    for (predictions, word_ids) in zip(batch, word_id_batch):
        # TODO probably no need to store mask?
        mask = word_ids != -1
        print(word_ids)
        word_ids = word_ids[mask]
        predictions = predictions[mask]
        print(predictions)
        unique_word_ids, word_id_counts = torch.unique_consecutive(word_ids, return_counts=True)
        agg_predictions = torch.zeros((len(unique_word_ids), predictions.shape[-1]), dtype=float)
        start_id = 0
        for i, (unique_word_id, word_id_count) in enumerate(zip(unique_word_ids, word_id_counts)):
            end_id = start_id + word_id_count
            agg_predictions[i] = predictions[start_id: end_id]
            start_id = end_id
        print(agg_predictions)
    return agg_predictions

In [58]:
word_ids()

torch.Size([2, 2, 3])